In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from Scratch.DataEncoders import CategoricalEncoder
from Scratch.DataEncoders import MinMaxNormalizer
from Scratch.Metrics import accuracy
from Scratch.KNNClassifier import KnnClassifier

### Data encoding

In [2]:
data = pd.read_csv("../datasets/stroke.csv")
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,No,Yes,Yes,Private,Urban,228.69,36.6,formerly smoked,Yes
1,Male,80.0,No,Yes,Yes,Private,Rural,105.92,32.5,never smoked,Yes
2,Female,49.0,No,No,Yes,Private,Urban,171.23,34.4,smokes,Yes
3,Female,79.0,Yes,No,Yes,Self-employed,Rural,174.12,24.0,never smoked,Yes
4,Male,81.0,No,No,Yes,Private,Urban,186.21,29.0,formerly smoked,Yes


In [3]:
enc = CategoricalEncoder()
cat_atrbs = ['gender','hypertension','heart_disease','ever_married','work_type','Residence_type','smoking_status']
data_enc = enc.fit_transform_df(data,cat_atrbs)

data_enc.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.6,0,Yes
1,1,80.0,0,1,1,2,0,105.92,32.5,1,Yes
2,0,49.0,0,0,1,2,1,171.23,34.4,2,Yes
3,0,79.0,1,0,1,3,0,174.12,24.0,1,Yes
4,1,81.0,0,0,1,2,1,186.21,29.0,0,Yes


In [4]:
scaler = MinMaxNormalizer()
atrbs = cat_atrbs + ['age','avg_glucose_level','bmi']
data_enc = scaler.fit_transform_df(data_enc,atrbs)
X = data_enc.iloc[:,0:-1]
y = data_enc['stroke']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,stratify=y)

### With my implementation

In [5]:
clsf = KnnClassifier(X_train,y_train,3,metric='manhattan')

In [6]:
preds = clsf.predict_all(X_test)

### With SkLearn's implementation

In [7]:
clsf2 = KNeighborsClassifier(3,metric='manhattan')

In [8]:
clsf2.fit(X_train,y_train)

KNeighborsClassifier(metric='manhattan', n_neighbors=3)

In [9]:
preds2 = clsf2.predict(X_test)

### Metrics

In [11]:
print(len(preds2[preds!=preds2]))
print(len(X_test))

0
685


In [12]:
accuracy(y_test,preds)

0.9357664233576642

### Conclusions

This is so first method i could not ignore efficiency. Without transforming everything to np arrays, it was painfully slow. My implementation is still slowler than skLearn's, but that's no surprise. 

However, resultwise my implementation is very good. Compared to skLearn's one, when using manhattan distance, all the predictions are the same. When using euclidena distance, 11/685 are wrong. That's still a 98.39% accuracy (relative to sklearn, not to the actual data), pretty good.

In conclusion, this method is easy to implement and surprisingly effective, but it's not very efficient, 
even whith improved implementations 